In [70]:
import torch

def sample_data(w,b,sample_size):
    x=torch.normal(0,1,(sample_size,len(w)))
    y=torch.matmul(x,w)+b
    y+=torch.normal(0,0.1,y.shape)
    return x,y.reshape((-1,1))

sample_size=1000
tw=torch.tensor([-2.0,3.0])
tb=torch.tensor(10)
tx,ty=sample_data(tw,tb,sample_size)

In [47]:
import random

def loss_fn(y_hat,y):
    #print(y_hat.shape)
    return (y_hat-y.reshape(y_hat.shape))**2/2

def line_fn(x,w,b):
    return torch.matmul(x,w)+b

def data_iter(batch_size,x,y):
    num_examples=len(x)
    indeces=list(range(num_examples))
    random.shuffle(indeces)
    for i in range(0,num_examples,batch_size):
        index_batch=indeces[i:min(i+batch_size,num_examples)]
        yield x[index_batch],y[index_batch]

def sgd(batch_size,lr,w,b):
    with torch.no_grad():
        w-=w.grad/batch_size
        w.grad.zero_()
        b-=b.grad/batch_size
        b.grad.zero_()
            

In [48]:
lr=0.03
batch_size=10
ep=3
iw=torch.normal(0,0.01,(2,1),requires_grad=True)
#iw=torch.zeros(2,requires_grad=True)
ib=torch.zeros(1,requires_grad=True)

for i in range(ep):
    for bx,by in data_iter(batch_size,tx,ty):
        loss=loss_fn(line_fn(tx,iw,ib),ty)
        loss.sum().backward()
        sgd(len(tx),lr,iw,ib)
    with torch.no_grad():
        lossd=loss_fn(line_fn(tx,iw,ib),ty)
        print(f"ep:{i},loss:{lossd.mean()} iw:{iw} ib:{ib}")


ep:0,loss:0.004595113918185234 iw:tensor([[-1.9989],
        [ 2.9943]], requires_grad=True) ib:tensor([9.9967], requires_grad=True)
ep:1,loss:0.004595113918185234 iw:tensor([[-1.9989],
        [ 2.9943]], requires_grad=True) ib:tensor([9.9967], requires_grad=True)
ep:2,loss:0.004595113918185234 iw:tensor([[-1.9989],
        [ 2.9943]], requires_grad=True) ib:tensor([9.9967], requires_grad=True)


In [71]:
from torch.utils import data
from torch import nn

nnsize=1000
nw=torch.tensor([-2.0,3.0])
nb=torch.tensor(5.0)
bz=10
nnx,nny=sample_data(nw,nb,nnsize)

def dataloader_iter(datas,batch_size,shuffle):
    dataset=data.TensorDataset(*datas)
    return data.DataLoader(dataset,batch_size,shuffle)

net=nn.Sequential(nn.Linear(2,1))

net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

loss=nn.MSELoss()

trainer=torch.optim.SGD(net.parameters(),lr=0.03)

repeat=3
for i in range(repeat):
    for sx,sy in dataloader_iter((nnx,nny),bz,True):
        l=loss(net(sx),sy)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    tl=loss(net(nnx),nny)
    print(f'i:{i} tl:{tl}')

i:0 tl:0.00920788012444973
i:1 tl:0.009206287562847137
i:2 tl:0.009178058244287968
